In [ ]:
%%capture
!pip install atomate2[strict]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

First we load makers from atomate2. Makers can be used to generate Flow objects with default values in a convenient way.

In [ ]:
#from atomate2.vasp.jobs.core import RelaxMaker, StaticMaker, RelaxSetGenerator
from atomate2.forcefields.jobs import CHGNetRelaxMaker, CHGNetStaticMaker, M3GNetRelaxMaker, M3GNetStaticMaker, GAPRelaxMaker, GAPStaticMaker


Then, we need standard classes, decorators, functions from jobflow

In [ ]:
from jobflow import job, Response, Flow, run_locally, Maker
from dataclasses import dataclass

We define a job to create a flow including constant volume optimizations

In [ ]:
@job
def get_energies(structure, volume_increments=[0.95,0.97,0.99, 1.00, 1.01, 1.03, 1.05 ]):
    jobs=[]
    volumes=[]
    energies=[]
    for inc in volume_increments:
        new_structure=structure.copy()
        new_structure.scale_lattice(structure.volume*inc)
        # job=RelaxMaker(input_set_generator=RelaxSetGenerator(user_incar_settings={"ISIF": 4})).make(new_structure)
        new_job=M3GNetRelaxMaker(relax_cell=False, relax_kwargs={"fmax": 0.0001}).make(new_structure)
        jobs.append(new_job)
        volumes.append(new_job.output.structure.volume)
        energies.append(new_job.output.output.energy)
    return Response(replace=Flow(jobs, output={"energies": energies, "volumes": volumes}))


We then compute the ev curve in a job starting from a list of energies, volumes

In [ ]:
from ase.units import kJ
from ase.eos import EquationOfState

@job
def get_results_ev_curve(list_volumes, list_energies):
    eos = EquationOfState(list_volumes, list_energies)
    v0, e0, B = eos.fit()
    eos.plot('eos.png')

    results = {}
    results["V0"] = v0
    results["e0"] = e0
    results["B"] = B

    return results

We define a Maker for convenience

In [ ]:
@dataclass
class EVMaker(Maker):
    name="ev"
    def make(structure):

        job1=new_job=M3GNetRelaxMaker(relax_cell=True, relax_kwargs={"fmax": 0.0001}).make(structure=structure)
        job2=get_energies(job1.output.structure)
        job3=get_results_ev_curve(list_volumes=job2.output["volumes"], list_energies=job2.output["energies"])
        return Flow([job1,job2,job3], job3.output)

Start the workflow

In [ ]:
from pymatgen.core import Structure
structure = Structure(
    lattice=[[0, 2.73, 2.73], [2.73, 0, 2.73], [2.73, 2.73, 0]],
    species=["Si", "Si"],
    coords=[[0, 0, 0], [0.25, 0.25, 0.25]],
)
print(structure.volume)
flow=EVMaker.make(structure=structure)

Then, we run it.

In [ ]:
response=run_locally(flow)

In [ ]:
print(response[flow[2].uuid][1].output)